In [1]:
import pandas as pd

In [ ]:
# 1. Mark each prediction as ambiguous
df['is_ambiguous'] = df['prediction_set'].apply(lambda labels: 1 if len(labels) > 1 else 0)

In [ ]:
# 2. Compute the fraction of ambiguous predictions for each combination of (universe decisions + subgroup)
group_cols = ['feature_set', 'model', 'threshold_policy', 'gender', 'nationality']
fractions = df.groupby(group_cols)['is_ambiguous'].mean().reset_index()

In [ ]:
# `fractions` now contains one row per (feature_set, model, threshold_policy, gender, nationality) 
# with the mean of `is_ambiguous`, which is the fraction of predictions that are ambiguous in that subgroup.
print(fractions.head())  # For debugging: view the first few rows of fraction results